In [1]:
import requests
from bs4 import BeautifulSoup
import re #regular expressions
import time #for delaying the while loop

In [2]:
#Sending the request
request=requests.get('https://www.zlatestranky.cz/firmy/rubrika/Restaurace/kraj/Hlavn%C3%AD%20m%C4%9Bsto%20Praha')
print(request.status_code)

200


In [3]:
#Making a soup
soup=BeautifulSoup(request.content) #why use .content instead of .text?
#Extracting restaurant titles
titles_soup=soup.find_all('h3')
titles=[]
for i in titles_soup:
    titles.append(i.find('a').text)

print(titles[:10])

['Repre restaurant', 'Restaurace Nad Ledem', 'SRI LANKAN CURRY HOUSE', 'Restaurace HOOTERS Vodičkova', '1. MEXICKÁ RESTAURACE', 'A.H.W. a.s.', 'Abri s.r.o.', 'ABSOLUT s.r.o.', 'ADAMOVÁ IVA', 'Adeta Service, s.r.o.']


In [6]:
#Extracting the link for each restaurant
links=[]
for i in titles_soup:
    links.append(f'https://www.zlatestranky.cz{i.find("a")["href"]}')

print(links[:10])

['https://www.zlatestranky.cz/profil/H270008', 'https://www.zlatestranky.cz/profil/H2052564', 'https://www.zlatestranky.cz/profil/H2028609', 'https://www.zlatestranky.cz/profil/H2056923', 'https://www.zlatestranky.cz/profil/H952175', 'https://www.zlatestranky.cz/profil/T1488202', 'https://www.zlatestranky.cz/profil/H731735', 'https://www.zlatestranky.cz/profil/T678796', 'https://www.zlatestranky.cz/profil/H217233', 'https://www.zlatestranky.cz/profil/H1909419']


In [17]:
#Request for a single restaurant page

#H270008 #original one, well defined restaurant
#H1778453 #this rest. has more than one tel. number, less services
#H1391009 #has no link, opening hours, email
request_profil=requests.get('https://www.zlatestranky.cz/profil/H1391009')

In [18]:
#Extracting the district 
profil_soup=BeautifulSoup(request_profil.content)
#address=profil_soup.find('address').find('span').text
address=profil_soup.find('span',{'itemprop':'description'}).text
print(address)
re.search('Praha [0-9]{1,2}',address).group(0)

Štupartská 9/745, 110 00 Praha 1-Staré Městookres Hlavní město Praha, Hlavní město Praha


'Praha 1'

In [8]:
#Extracting the ratings
profil_soup.find('span',{'itemprop':'ratingValue'}).text

'0'

In [9]:
#Extracting the number of reviews
profil_soup.find('span',{'itemprop':'reviewCount'}).text

'0'

In [10]:
### Extracting the opening hours ###
opening_hours_table=profil_soup.find('table',{'class':'table table-condensed'}).find_all('td') 
opening_hours_table_text=[] #empty list
#extract only day and hours
for i in opening_hours_table:
    if re.match('Dnes|\n',i.text):
        pass
    else:
        opening_hours_table_text.append(i.text)
print(opening_hours_table_text)
#creating a dictionary (key=day,value=opening hours)
opening_hours_dict={}
for i in range(0,len(opening_hours_table_text),2):
    opening_hours_dict[opening_hours_table_text[i]]=opening_hours_table_text[i+1]
print(opening_hours_dict)

['Po', '10:30 - 23', 'Út', '10:30 - 23', 'St', '10:30 - 23', 'Čt', '10:30 - 23', 'Pá', '10:30 - 23', 'So', '12 - 23']
{'Po': '10:30 - 23', 'Út': '10:30 - 23', 'St': '10:30 - 23', 'Čt': '10:30 - 23', 'Pá': '10:30 - 23', 'So': '12 - 23'}


In [11]:
### Calculating the opening time span for each day ###
#Writing a function to convert the time span into a number
def time_span_to_number(time_span):
    hours=[time_span.split()[i] for i in [0,2]] #splitting by space and disregarding "-"
    start_end=[] #empty list for converted starting and ending values
    for i in hours:
        if re.search(':',i): #if the string is not a whole hour, convert it
            splits=i.split(':')
            start_end.append(float(splits[0])+float(splits[1])/60)
        else:
            start_end.append(float(i))
    return round(start_end[1] - start_end[0],2)

time_span_dict={key:time_span_to_number(value) for (key,value) in opening_hours_dict.items()}
print(time_span_dict)

{'Po': 12.5, 'Út': 12.5, 'St': 12.5, 'Čt': 12.5, 'Pá': 12.5, 'So': 11.0}


In [12]:
#requests for all pages
request_list=[]
i=1 #initializing the first iteration
indicator=200 #initializing the while loop
while indicator == 200:
    request=requests.get(f'https://www.zlatestranky.cz/firmy/rubrika/Restaurace/kraj/Hlavn%C3%AD%20m%C4%9Bsto%20Praha/{i}')
    indicator=request.status_code
    time.sleep(0.2)
    if indicator == 200:
        request_list.append(request.content)
        i += 1
    else:
        print(f'Successfuly requested {i-1} pages')

#Note: tested and works well, it tok me 22 second (i added a sleep delayer)        
#Note: added the loop initiator (it did not work in another file since request was not defined)        

Successfuly requested 22 pages


In [29]:
#Extracting the phone number

print(profil_soup.find('td', {'itemprop':'telephone'}).text) #finds only the first telephone

#Finding of all tel. numbers
#result is a dictionary {name of telephone: number}

if profil_soup.find('td', {'itemprop':'telephone'}).text == None:
    print(None)
else:
    telephone_numbers = []
    telephone_names = []

    #extracting telephone numbers:
    all_telephones = profil_soup.find_all('td', {'itemprop':'telephone'})
    for i in all_telephones:
        telephone_numbers.append(i.text)

    #extracting telephone names:
    telephone_names = []
    for i in telephone_numbers:
        telephone_names.append(profil_soup.find('td', text=i).find_next_sibling('td').text)

    telephones = {telephone_names[i]: telephone_numbers[i] for i in range(len(telephone_numbers))}

    print(telephone_numbers) #just for verification
    print(telephone_names) #just for verification
    print(telephones)
    


+420 222 314 308
['+420 222 314 308', '+420 222 315 185']
['hlavní telefon', 'další telefon']
{'hlavní telefon': '+420 222 314 308', 'další telefon': '+420 222 315 185'}


In [14]:
#Extracting email adress
print(profil_soup.find('a', {'data-ta':'EmailClick'}).text)

#similiarly as in the case of telephones, could be email_dict={'name of email':'email'}
#but it looks like there usually is only one email, so perhaps redundant

rezervace@repre-restaurant.cz


In [15]:
#Extracting link for the restaurant's web page

if profil_soup.find('a', {'data-ta':'LinkClick'}) == None: #condition, that is useful when the restaurant has no web
    print(None)
else:
    print(profil_soup.find('a', {'data-ta':'LinkClick'})['href']) #simply printing the link


http://www.repre-restaurant.cz


In [16]:
#Extracting payment methods
list_payment_methods_raw = profil_soup.find('h2', text='Platební metody').find_next_sibling('ul', {'class':'list-inline'}).find_all('li') #finding list of payment methods (wraped in <li><\li>)
list_payment_methods = [] #empty list to be used in the next for loop

for i in list_payment_methods_raw: #unwraping the <li>
    list_payment_methods.append(i.text)

print(list_payment_methods) #list containing strings, representing individual payment methods

['faktura', 'hotově', 'VISA', 'MASTERCARD', 'CHEQUEDEJEUNER', 'TICKETRESTAURANT', 'SODEXO', 'American Express']


In [76]:
#Extracting the Services (and other) tags
services_raw = profil_soup.find('div', {'class':'col-sm-12 tagcloud'}).find_all() #finding all children of the panel "Produkty+Služby+Značky"
services = []

for i in services_raw:
    services.append(i.text)

print(services) #list of all three categories together

#Separating by categories
#services_prodcuts = services[services.index('Produkty')+1:services.index('Služby')]# previously separated manually like this
#now coded dynamically further

['Produkty', 'taneční parket', 'Služby', 'česká kuchyně', 'mezinárodní kuchyně', 'nekuřácký prostor', 'Značky', 'salónek', 'klimatizace', 'bezbariérový přístup']


In [74]:
#Just for verifying that the code below works (the order was mixed)
#services = ['Značky', 'salónek', 'klimatizace', 'bezbariérový přístup', 'Produkty', 'taneční parket', 'Služby', 'česká kuchyně', 'mezinárodní kuchyně', 'nekuřácký prostor']

In [55]:
#Defining function for separating Servicies by category:
def services_separator(category):
    '''This function separates list of all services names and items into particular item-sets

     Args: The function expects imput category to be equal one of the three: 'Produkty', 'Služby', 'Značky'

     Other info: The function assumes that list services containing all services and items together already exists.
                 Additionaly, it assumes that there are at most these three categories of services
                 In case we discover more categories, they would have to be added manually.
    '''
    try:
        ##defining the helper list of positions of categories
        services_index_list = []

        for i in ['Produkty', 'Služby', 'Značky']: #<-- in case of more categories, add here!
            if i in services:
                services_index_list.append(services.index(i))     

        services_index_list.sort()

        ##selecting particular items
        if services.index(category) == max(services_index_list):
            services_category = services[services.index(category)+1:] #case when the given category is last in the services list
        else:
            services_category = services[services.index(category)+1:services_index_list[services_index_list.index(services.index(category))+1]]
            #when the given category is not last, subset the part of services list between the given category and the next category
    
    except ValueError: #In case that retaurant does not have given category specified, None will return
        return(None)

    else: #when the category is specified, the items will return
        return(services_category)

In [77]:
#The function defined above put to use, just for example
services_prodcuts = services_separator('Produkty')
services_services = services_separator('Služby')
services_marks = services_separator('Značky')
print(services_prodcuts) 
print(services_services) 
print(services_marks) 

['taneční parket']
['česká kuchyně', 'mezinárodní kuchyně', 'nekuřácký prostor']
['salónek', 'klimatizace', 'bezbariérový přístup']


In [6]:
#Extracting restaurants name
restaurant_name=profil_soup.find('h1',{'itemprop':'name'}).text
print(restaurant_name) 

Repre restaurant
